# Machine Learning Engineer Nanodegree
## Capstone Project
Ian Gregson
September 22nd 2018

# 1. Definition

### Project Overview

In this section, look to provide a high-level overview of the project in layman’s terms. Questions to ask yourself when writing this section:
- _Has an overview of the project been provided, such as the problem domain, project origin, and related datasets or input data?_
- _Has enough background information been given so that an uninformed reader would understand the problem domain and following problem statement?_


The domain background for this project is that it comes directly from my work for a startup that is building a customer data platform that helps users maximize their account based recurring revenue. In this field, BizOps profressional leverage large quantities of data from multiple sources such as CRMs or customer engagement analytics tools. The platform we build collates thiss data to provide a single view of an account (a non trivial problem in this space) but much of the insights that are gleaned from this large quantity of data come from the judgements of BizOps professionals studying the data.

The project seeks to take steps into a new way of deriving insights from this large suite of structured data. By automating some aspects of the data analysis with machine learning models will free up the professionals to apply their extensive domain expertise in solving new problems.

I have a personal motivation in undertaking this project because I want to help push my team to become leaders in our field.

### Problem Statement

In this section, you will want to clearly define the problem that you are trying to solve, including the strategy (outline of tasks) you will use to achieve the desired solution. You should also thoroughly discuss what the intended solution will be for this problem. Questions to ask yourself when writing this section:
- _Is the problem statement clearly defined? Will the reader understand what you are expecting to solve?_
- _Have you thoroughly discussed how you will attempt to solve the problem?_
- _Is an anticipated solution clearly defined? Will the reader understand what results you are looking for?_


The problem to be solved in this project is the problem of lead, opportunity and account scoring. There is historical data that shows which leads, opportunities and accounts eventually become accounts with annually recurring revenue. BizOps practitioners will use this historical data to make judgements on what active leads, oppoortunities and accounts are most likely to convert by comparing their features with the features of examples from the historical data.

This project seeks to build a machine learning model that will make these judgements.

To build that model, this project will work through the following steps:

* Establish and prepare evaluation metrics
* Explore the dataset and establish necessary pre-processing steps
* Establish a benchmark using a dummy classifier
* Carry out pre-processing of the dataset
* Implement a number of models using grid search to establish the best hyperparameters
* Train and validate the models on the dataset
* Refine the model that looks to have the best performance
* Evaluate the performance of the model on the dataset against the benchmark and establish whether or not the model is viable for further use

The model will be deemed acceptable for furhter use if the predictions it makes are better than those returned by the dummy classifier.


### Metrics

In this section, you will need to clearly define the metrics or calculations you will use to measure performance of a model or result in your project. These calculations and metrics should be justified based on the characteristics of the problem and problem domain. Questions to ask yourself when writing this section:
- _Are the metrics you’ve chosen to measure the performance of your models clearly discussed and defined?_
- _Have you provided reasonable justification for the metrics chosen based on the problem and solution?_


Since this project aims to solve a classification problem, I expect that Confusion Matrix and F1 Score metric will provide suitable evaluation of the performance of the benchmark and solution models.

The Confusion Matrix will provide a high level indication of how each of the implemented models are performing. This will be useful in establishing which model should be refined.

The F1 Score will provide a numeric means of judging the performance that can help more definiteively assess the performance of final model and ascertain whether or not it can be used further for Account, Lead and Opportunity Scoring.

## 2. Analysis

### Data Exploration

In this section, you will be expected to analyze the data you are using for the problem. This data can either be in the form of a dataset (or datasets), input data (or input files), or even an environment. The type of data should be thoroughly described and, if possible, have basic statistics and information presented (such as discussion of input features or defining characteristics about the input or environment). Any abnormalities or interesting qualities about the data that may need to be addressed have been identified (such as features that need to be transformed or the possibility of outliers). Questions to ask yourself when writing this section:
- _If a dataset is present for this problem, have you thoroughly discussed certain features about the dataset? Has a data sample been provided to the reader?_
- _If a dataset is present for this problem, are statistics about the dataset calculated and reported? Have any relevant results from this calculation been discussed?_
- _If a dataset is **not** present for this problem, has discussion been made about the input space or input data for your problem?_
- _Are there any abnormalities or characteristics about the input space or dataset that need to be addressed? (categorical variables, missing values, outliers, etc.)_



In [1]:
# Let's load up our std libraries and start having some fun

import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# First, we'll take a look at the opporunities dataset
!ls data/csv

DataStore_query-1536516138031.json.csv


In [5]:
raw_opportunities_data = pd.read_csv('data/csv/DataStore_query-1536516138031.json.csv', low_memory=False)
display(raw_opportunities_data.shape)
display(raw_opportunities_data.columns)

(5809, 471)

Index(['Labels', 'Name', 'CreatedDate', 'CloseDate', 'Amount', 'StageName',
       'Probability', 'Type', 'AccountId', 'Account_Region__c',
       ...
       'What_will_developers_need_to_succeed__c', 'Who_cares_about_it__c',
       'Who_is_leading_the_sale__c', 'Who_is_the_competition__c',
       'Who_needs_to_be_convinced_of_what_for_se__c',
       'Why_does_this_opp_merit_more_investment__c',
       'Why_should_the_customer_change__c',
       'Why_should_they_work_with_MuleSoft__c', 'Winning_Competitor__c',
       'Id.1'],
      dtype='object', length=471)

In [12]:
# Filter down the dataset to the labels we are interested in: Closed Won and Closed Lost
raw_opportunities_data = raw_opportunities_data[raw_opportunities_data['StageName'].isin(['Closed Won', 'Closed Lost'])]
row_count, col_count = raw_opportunities_data.shape

# drop columns where less than 80% of the rows have values
threshold = int(round(col_count * 0.8))
data = raw_opportunities_data.dropna(thresh=threshold, axis='columns')
display(data.shape)

(1519, 196)

In [13]:
display(raw_opportunities_data[['CloseDate', 'StageName']].describe())
display(raw_opportunities_data[['CloseDate', 'StageName']].sample(10))

,CloseDate,StageName
count,1519,1519
unique,120,2
top,2018-07-31,Closed Won
freq,84,1127


,CloseDate,StageName
5345,2018-07-04,Closed Lost
5603,2018-08-10,Closed Lost
5392,2018-12-21,Closed Won
3637,2018-06-29,Closed Won
5137,2018-07-30,Closed Won
1056,2018-09-04,Closed Lost
1874,2018-08-22,Closed Won
295,2018-08-31,Closed Lost
3372,2018-08-15,Closed Lost
5301,2018-09-05,Closed Lost


`StageName` is the label column. Of 1519 rows, 1127 are Opportunities that were won while only 392 were lost. This leads to suspicion that the model that will be built here is likely to be biased to create false positives. Also, it means that if our model were to guess that every row would be 'Closed Won', 74% of the predictions would be correct.

In [16]:
data.to_csv('data/data.csv')

In [18]:
# Next step of data pre-processing 
# Drop columns with long form text, urls, IDs, etc
cols_to_remove = [
    'Id.1',
    'Why_does_this_opp_merit_more_investment__c',
    'What_upfront_contract_was_set_for_call__c',
    'What_pain_areas_should_we_dig_deeper_on__c',
    'What_is_the_use_case_s__c',
    'What_is_the_business_need_initiative__c',
    'What_initiatives_will_we_support__c',
    'What_does_prospect_think_problem_is__c',
    'What_did_we_learn_on_the_intro_call__c',
    'Strategy_Choices__c',
    'Stage_Choices__c',
    'Stage__c',
    'Specific_Next_Step__c',
    'Parent_Opportunity_Id_18_digit__c',
    'Parent_Opportunity__c',
    'Original_Quote__c',
    'Opportunity_Update_Weekly__c',
    'mkto_si__MarketoAnalyzer__c',
    'Lightbulb_moment_POV_to_deliver_on_call__c',
    'Late_Breaking_News__c',
    'Info_evidence_for_POV__c',
    'If_Closed_Lost_Primary_Reason__c',
    'Id',
    'DM_Close_Type__c',
    'CreatedById',
    'CampaignId',
    'AccountId',
    'Labels',
    'RecordTypeId',
    'Pricebook2Id',
    'Parent_Opportunity_Id_18_digit__c',
    'Parent_Opportunity__c',
    'OwnerId',
    'Opportunity_Owner_ID_18_digit__c',
    'Opportunity_ID_18_digit__c',
    'Lead_Passed_By__c',
    'LastModifiedById',
]
data = data.drop(columns=cols_to_remove)
data.to_csv('data/csv/data.csv')

### Exploratory Visualization

In this section, you will need to provide some form of visualization that summarizes or extracts a relevant characteristic or feature about the data. The visualization should adequately support the data being used. Discuss why this visualization was chosen and how it is relevant. Questions to ask yourself when writing this section:
- _Have you visualized a relevant characteristic or feature about the dataset or input data?_
- _Is the visualization thoroughly analyzed and discussed?_
- _If a plot is provided, are the axes, title, and datum clearly defined?_


### Algorithms and Techniques

In this section, you will need to discuss the algorithms and techniques you intend to use for solving the problem. You should justify the use of each one based on the characteristics of the problem and the problem domain. Questions to ask yourself when writing this section:
- _Are the algorithms you will use, including any default variables/parameters in the project clearly defined?_
- _Are the techniques to be used thoroughly discussed and justified?_
- _Is it made clear how the input data or datasets will be handled by the algorithms and techniques chosen?_


### Benchmark

In this section, you will need to provide a clearly defined benchmark result or threshold for comparing across performances obtained by your solution. The reasoning behind the benchmark (in the case where it is not an established result) should be discussed. Questions to ask yourself when writing this section:
- _Has some result or value been provided that acts as a benchmark for measuring performance?_
- _Is it clear how this result or value was obtained (whether by data or by hypothesis)?_


## 3. Methodolgy

### Data Preprocessing

In this section, all of your preprocessing steps will need to be clearly documented, if any were necessary. From the previous section, any of the abnormalities or characteristics that you identified about the dataset will be addressed and corrected here. Questions to ask yourself when writing this section:
- _If the algorithms chosen require preprocessing steps like feature selection or feature transformations, have they been properly documented?_
- _Based on the **Data Exploration** section, if there were abnormalities or characteristics that needed to be addressed, have they been properly corrected?_
- _If no preprocessing is needed, has it been made clear why?_


### Implementation

In this section, the process for which metrics, algorithms, and techniques that you implemented for the given data will need to be clearly documented. It should be abundantly clear how the implementation was carried out, and discussion should be made regarding any complications that occurred during this process. Questions to ask yourself when writing this section:
- _Is it made clear how the algorithms and techniques were implemented with the given datasets or input data?_
- _Were there any complications with the original metrics or techniques that required changing prior to acquiring a solution?_
- _Was there any part of the coding process (e.g., writing complicated functions) that should be documented?_


### Refinement

In this section, you will need to discuss the process of improvement you made upon the algorithms and techniques you used in your implementation. For example, adjusting parameters for certain models to acquire improved solutions would fall under the refinement category. Your initial and final solutions should be reported, as well as any significant intermediate results as necessary. Questions to ask yourself when writing this section:
- _Has an initial solution been found and clearly reported?_
- _Is the process of improvement clearly documented, such as what techniques were used?_
- _Are intermediate and final solutions clearly reported as the process is improved?_


## 4. Results

### Model Evaluation and Validation

In this section, the final model and any supporting qualities should be evaluated in detail. It should be clear how the final model was derived and why this model was chosen. In addition, some type of analysis should be used to validate the robustness of this model and its solution, such as manipulating the input data or environment to see how the model’s solution is affected (this is called sensitivity analysis). Questions to ask yourself when writing this section:
- _Is the final model reasonable and aligning with solution expectations? Are the final parameters of the model appropriate?_
- _Has the final model been tested with various inputs to evaluate whether the model generalizes well to unseen data?_
- _Is the model robust enough for the problem? Do small perturbations (changes) in training data or the input space greatly affect the results?_
- _Can results found from the model be trusted?_


### Jusitifcation

In this section, your model’s final solution and its results should be compared to the benchmark you established earlier in the project using some type of statistical analysis. You should also justify whether these results and the solution are significant enough to have solved the problem posed in the project. Questions to ask yourself when writing this section:
- _Are the final results found stronger than the benchmark result reported earlier?_
- _Have you thoroughly analyzed and discussed the final solution?_
- _Is the final solution significant enough to have solved the problem?_


## 5. Conclusion

### Free-Form Visualization

In this section, you will need to provide some form of visualization that emphasizes an important quality about the project. It is much more free-form, but should reasonably support a significant result or characteristic about the problem that you want to discuss. Questions to ask yourself when writing this section:
- _Have you visualized a relevant or important quality about the problem, dataset, input data, or results?_
- _Is the visualization thoroughly analyzed and discussed?_
- _If a plot is provided, are the axes, title, and datum clearly defined?_


### Reflection

In this section, you will summarize the entire end-to-end problem solution and discuss one or two particular aspects of the project you found interesting or difficult. You are expected to reflect on the project as a whole to show that you have a firm understanding of the entire process employed in your work. Questions to ask yourself when writing this section:
- _Have you thoroughly summarized the entire process you used for this project?_
- _Were there any interesting aspects of the project?_
- _Were there any difficult aspects of the project?_
- _Does the final model and solution fit your expectations for the problem, and should it be used in a general setting to solve these types of problems?_


### Improvement

In this section, you will need to provide discussion as to how one aspect of the implementation you designed could be improved. As an example, consider ways your implementation can be made more general, and what would need to be modified. You do not need to make this improvement, but the potential solutions resulting from these changes are considered and compared/contrasted to your current solution. Questions to ask yourself when writing this section:
- _Are there further improvements that could be made on the algorithms or techniques you used in this project?_
- _Were there algorithms or techniques you researched that you did not know how to implement, but would consider using if you knew how?_
- _If you used your final solution as the new benchmark, do you think an even better solution exists?_


--------------------------------------------------------------------------

**Before submitting, ask yourself. . .**

- Does the project report you’ve written follow a well-organized structure similar to that of the project template?
- Is each section (particularly **Analysis** and **Methodology**) written in a clear, concise and specific fashion? Are there any ambiguous terms or phrases that need clarification?
- Would the intended audience of your project be able to understand your analysis, methods, and results?
- Have you properly proof-read your project report to assure there are minimal grammatical and spelling mistakes?
- Are all the resources used for this project correctly cited and referenced?
- Is the code that implements your solution easily readable and properly commented?
- Does the code execute without error and produce results similar to those reported?
